# <font color=blue>ASSIGNMENT
In this assignment, you are going to measure the performance of the model you created with the Titanic dataset in the previous lesson. To complete this assignment, send a link to a Jupyter notebook containing solutions to the following tasks.


1-Evaluate your model's performance with cross validation and using different metrics.

  

In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv("E:/user/Notebooks/data/titanic/train.csv", low_memory=False)

In [25]:
from sklearn.model_selection import cross_validate, cross_val_score

In [26]:
y=data.Survived
data["is_male"] = pd.get_dummies(data.Sex, drop_first=True)
X=data[['Pclass','is_male','Fare','SibSp','Parch']]

In [27]:
log_reg_model = LogisticRegression()

cv = cross_validate(estimator=log_reg_model,
                    X=X,
                    y=y,
                    cv=10,
                    return_train_score=True,
                   )

print('Train Scores :', cv['train_score'], sep = '\n')
print("-"*50)
print('Test Scores     :', cv['test_score'], sep = '\n')

Train Scores :
[0.80399501 0.79551122 0.8042394  0.79177057 0.79551122 0.79925187
 0.79800499 0.80174564 0.7967581  0.80299252]
--------------------------------------------------
Test Scores     :
[0.81111111 0.78651685 0.7752809  0.83146067 0.78651685 0.7752809
 0.7752809  0.7752809  0.83146067 0.78651685]


In [28]:
print('Mean score of train set : ', cv['train_score'].mean())
print('Mean score of test set  : ', cv['test_score'].mean())

Mean score of train set :  0.7989780542401798
Mean score of test set  :  0.7934706616729088


In [29]:
cv = cross_validate(estimator=log_reg_model,
                    X=X,
                    y=y,
                    cv=10,
                    return_train_score=True,
                    scoring = ['accuracy', 'precision', 'r2','recall']
                   )

In [30]:
print('Train Set Mean Accuracy  : {:.2f}  '.format(cv['train_accuracy'].mean()))
print('Train Set Mean R-square  : {:.2f}  '.format(cv['train_r2'].mean()))
print('Train Set Mean Precision : {:.2f}'.format(cv['train_precision'].mean()))
print('Train Set Mean recall : {:.2f}\n'.format(cv['train_recall'].mean()))

print('Test Set Mean Accuracy   : {:.2f}  '.format(cv['test_accuracy'].mean()))
print('Test Set Mean R-square   : {:.2f}  '.format(cv['test_r2'].mean()))
print('Test Set Mean Precision  : {:.2f}  '.format(cv['test_precision'].mean()))
print('Test Set Mean recall  : {:.2f}  '.format(cv['test_recall'].mean()))

Train Set Mean Accuracy  : 0.80  
Train Set Mean R-square  : 0.15  
Train Set Mean Precision : 0.77
Train Set Mean recall : 0.68

Test Set Mean Accuracy   : 0.79  
Test Set Mean R-square   : 0.13  
Test Set Mean Precision  : 0.76  
Test Set Mean recall  : 0.68  


 2-Determine the model with the most appropriate parameters by hyperparameter tuning.

In [31]:
parameters = {"C": [10 ** x for x in range (-5, 5, 1)],
              "penalty": ['l1', 'l2']
             }

In [32]:
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression()
grid_cv = GridSearchCV(estimator=log_reg,
                       param_grid = parameters,
                       cv = 10
                      )

grid_cv.fit(X, y)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l1', 'l2']})

In [33]:
print("Best Parameters : ", grid_cv.best_params_)
print("Best Score      : ", grid_cv.best_score_)

Best Parameters :  {'C': 1, 'penalty': 'l2'}
Best Score      :  0.7934706616729088


In [34]:
results = grid_cv.cv_results_

df = pd.DataFrame(results)

In [35]:
df = df[['param_penalty','param_C', 'mean_test_score']]
df = df.sort_values(by='mean_test_score', ascending = False)
df

,param_penalty,param_C,mean_test_score
11,l2,1,0.793471
13,l2,10,0.793471
15,l2,100,0.793471
17,l2,1000,0.793471
19,l2,10000,0.793471
9,l2,0.1,0.791223
7,l2,0.01,0.718377
5,l2,0.001,0.677990
1,l2,1e-05,0.666729
3,l2,0.0001,0.663383


In [36]:
from sklearn.model_selection import RandomizedSearchCV

rs_cv = RandomizedSearchCV(estimator=log_reg,
                           param_distributions = parameters,
                           cv = 10,
                           n_iter = 10,
                           random_state = 111,
                           scoring = 'precision'
                      )

rs_cv.fit(X, y)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(),
                   param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1,
                                              1, 10, 100, 1000, 10000],
                                        'penalty': ['l1', 'l2']},
                   random_state=111, scoring='precision')

In [38]:
print("Best parameters      : ", rs_cv.best_params_, "\n")
print("Best precision value : ", rs_cv.best_score_)

Best parameters      :  {'penalty': 'l2', 'C': 10} 

Best precision value :  0.7619446598756944


GridSearchCV gave better result of score